# 📊 Análisis Exploratorio de Datos (EDA)
## Clasificación de Niveles de Obesidad - Regresión Ordinal

Este notebook realiza un análisis exploratorio completo del dataset para:
1. Entender la estructura y características de los datos
2. Analizar la distribución de las clases de obesidad
3. Identificar problemas como desbalance de clases
4. Generar visualizaciones y reportes

**Problema**: Regresión Ordinal con 7 clases ordenadas de niveles de obesidad


## 1. Importación de Librerías

### ¿Por qué estas librerías?

- **pandas**: Manipulación y análisis de datos estructurados. Permite trabajar con DataFrames de forma eficiente.
- **numpy**: Operaciones matemáticas y estadísticas. Base para muchas operaciones de ML.
- **matplotlib y seaborn**: Creación de visualizaciones. Seaborn facilita gráficos estadísticos atractivos.
- **warnings**: Para suprimir mensajes innecesarios y mantener la salida limpia.

### Configuración de Visualización

Configuramos el estilo de los gráficos para que sean más legibles y profesionales.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

# Configuración para mejorar la visualización
warnings.filterwarnings('ignore')
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

print("✓ Librerías importadas correctamente")


## 2. Carga del Dataset

### ¿Qué hace este paso?

Este paso carga el archivo CSV con los datos originales. Es importante verificar que:
- El archivo se carga correctamente
- No hay errores de formato
- La estructura es la esperada

### ¿Por qué es importante?

Antes de hacer cualquier análisis, debemos asegurarnos de que los datos están en el formato correcto y que no hay problemas obvios (valores faltantes masivos, columnas incorrectas, etc.).


In [ ]:
# Cargar el dataset
df = pd.read_csv('ObesityDataSet_raw_and_data_sinthetic.csv')

# Mostrar información básica del dataset
print(f"✓ Dataset cargado exitosamente")
print(f"  - Número de filas: {len(df)}")
print(f"  - Número de columnas: {len(df.columns)}")
print(f"\nPrimeras 5 filas del dataset:")
df.head()


In [ ]:
# Información general del dataset
print("Información general del dataset:")
df.info()


## 3. Identificación de la Variable Objetivo

### ¿Qué hace este paso?

Identificamos la columna que contiene las clases de obesidad (`NObeyesdad`), que es nuestra variable objetivo (lo que queremos predecir).

### ¿Por qué es importante?

- Debemos verificar que la variable objetivo existe
- Necesitamos conocer cuántas clases únicas hay
- Es el primer paso para entender el problema de clasificación

### Clases de Obesidad

El problema tiene 7 clases ordenadas (regresión ordinal):
1. Insufficient_Weight
2. Normal_Weight
3. Overweight_Level_I
4. Overweight_Level_II
5. Obesity_Type_I
6. Obesity_Type_II
7. Obesity_Type_III


In [ ]:
# La variable objetivo es 'NObeyesdad'
target_column = 'NObeyesdad'

# Verificar que la columna existe
if target_column in df.columns:
    print(f"✓ Variable objetivo encontrada: '{target_column}'")
    print(f"  - Tipo de datos: {df[target_column].dtype}")
    print(f"  - Valores únicos: {df[target_column].nunique()}")
    print(f"\nClases de obesidad encontradas:")
    print(df[target_column].unique())
else:
    print(f"✗ Error: La columna '{target_column}' no existe en el dataset")
    print(f"Columnas disponibles: {list(df.columns)}")


## 4. Análisis de la Distribución de Clases

### ¿Qué hace este paso?

Analizamos cuántos registros hay en cada clase de obesidad. Esto es crítico porque:

1. **Identifica desbalance**: Si una clase tiene muchos más registros que otra, el modelo puede sesgarse
2. **Planifica estrategias**: Si hay desbalance, necesitamos técnicas especiales (SMOTE, pesos de clase, etc.)
3. **Valida calidad de datos**: Distribuciones muy extrañas pueden indicar problemas en los datos

### Métricas de Desbalance

**Ratio de Desbalance (IR - Imbalance Ratio)**:
- Fórmula: `IR = tamaño_clase_mayoritaria / tamaño_clase_minoritaria`
- Interpretación:
  - IR < 2: Balanceado ✅
  - IR 2-5: Desbalanceo leve ⚠️
  - IR 5-10: Desbalanceo moderado ⚠️⚠️
  - IR > 10: Desbalanceo severo ❌

### ¿Por qué es importante para regresión ordinal?

Aunque el problema es ordinal (las clases tienen orden), el desbalance puede afectar:
- La capacidad del modelo de aprender clases minoritarias
- La precisión de las métricas de evaluación
- La necesidad de técnicas de balanceo


In [ ]:
# Contar la frecuencia de cada clase
distribucion = df[target_column].value_counts().sort_index()

# Calcular porcentajes
porcentajes = (df[target_column].value_counts(normalize=True) * 100).sort_index()

# Crear un DataFrame con la información
distribucion_df = pd.DataFrame({
    'Clase': distribucion.index,
    'Cantidad': distribucion.values,
    'Porcentaje': porcentajes.values
})

print("=" * 80)
print("DISTRIBUCIÓN DE CLASES DE OBESIDAD")
print("=" * 80)
print(distribucion_df.to_string(index=False))


In [ ]:
# Estadísticas adicionales
print("\n" + "=" * 80)
print("ESTADÍSTICAS DEL DESBALANCE")
print("=" * 80)
print(f"Total de registros: {len(df)}")
print(f"Número de clases: {len(distribucion)}")
print(f"Clase con MÁS registros: {distribucion.idxmax()} ({distribucion.max()} registros)")
print(f"Clase con MENOS registros: {distribucion.idxmin()} ({distribucion.min()} registros)")

# Calcular el ratio de desbalance (Imbalance Ratio - IR)
# IR = tamaño de la clase mayoritaria / tamaño de la clase minoritaria
clase_mayoritaria = distribucion.max()
clase_minoritaria = distribucion.min()
imbalance_ratio = clase_mayoritaria / clase_minoritaria

print(f"\nRatio de Desbalance (IR): {imbalance_ratio:.2f}")
print(f"  - Esto significa que la clase mayoritaria tiene {imbalance_ratio:.2f}x más registros")
print(f"  - que la clase minoritaria")

# Clasificar el nivel de desbalance
if imbalance_ratio < 2:
    nivel = "BALANCEADO"
elif imbalance_ratio < 5:
    nivel = "DESBALANCEO LEVE"
elif imbalance_ratio < 10:
    nivel = "DESBALANCEO MODERADO"
else:
    nivel = "DESBALANCEO SEVERO"

print(f"\nNivel de desbalance: {nivel}")


## 5. Visualizaciones de la Distribución

### ¿Qué hace este paso?

Creamos visualizaciones para entender mejor la distribución de clases. Las visualizaciones son importantes porque:

1. **Comunicación**: Es más fácil entender un gráfico que una tabla de números
2. **Identificación rápida**: Los problemas de desbalance se ven inmediatamente
3. **Presentación**: Necesarias para reportes y presentaciones

### Tipos de Gráficos que Crearemos

1. **Gráfico de Barras (Cantidad)**: Muestra el número absoluto de registros por clase
2. **Gráfico de Barras (Porcentajes)**: Muestra la proporción de cada clase
3. **Gráfico de Pastel**: Visualización alternativa que muestra proporciones
4. **Gráfico de Barras Horizontal**: Ordenado de menor a mayor para identificar fácilmente las clases minoritarias

### ¿Por qué múltiples visualizaciones?

Cada tipo de gráfico tiene sus ventajas:
- **Barras verticales**: Fáciles de leer, buenas para comparar valores
- **Barras horizontales**: Útiles cuando hay muchos valores o nombres largos
- **Pastel**: Intuitivo para ver proporciones, pero menos preciso para comparar valores exactos


In [ ]:
# Crear figura con subplots
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Análisis de Distribución de Clases de Obesidad', fontsize=16, fontweight='bold')

# 1. Gráfico de barras - Cantidad de registros por clase
ax1 = axes[0, 0]
distribucion.plot(kind='bar', ax=ax1, color='steelblue', edgecolor='black')
ax1.set_title('Distribución de Clases (Cantidad de Registros)', fontweight='bold')
ax1.set_xlabel('Clase de Obesidad', fontweight='bold')
ax1.set_ylabel('Cantidad de Registros', fontweight='bold')
ax1.tick_params(axis='x', rotation=45)
ax1.grid(axis='y', alpha=0.3)

# Agregar valores en las barras
for i, v in enumerate(distribucion.values):
    ax1.text(i, v + 10, str(v), ha='center', va='bottom', fontweight='bold')

# 2. Gráfico de barras - Porcentajes
ax2 = axes[0, 1]
porcentajes.plot(kind='bar', ax=ax2, color='coral', edgecolor='black')
ax2.set_title('Distribución de Clases (Porcentajes)', fontweight='bold')
ax2.set_xlabel('Clase de Obesidad', fontweight='bold')
ax2.set_ylabel('Porcentaje (%)', fontweight='bold')
ax2.tick_params(axis='x', rotation=45)
ax2.grid(axis='y', alpha=0.3)

# Agregar valores en las barras
for i, v in enumerate(porcentajes.values):
    ax2.text(i, v + 0.5, f'{v:.1f}%', ha='center', va='bottom', fontweight='bold')

# 3. Gráfico de pastel (pie chart)
ax3 = axes[1, 0]
colors = sns.color_palette("Set3", len(distribucion))
wedges, texts, autotexts = ax3.pie(distribucion.values, 
                                    labels=distribucion.index,
                                    autopct='%1.1f%%',
                                    colors=colors,
                                    startangle=90)
ax3.set_title('Distribución de Clases (Gráfico Circular)', fontweight='bold')

# Mejorar la legibilidad del gráfico de pastel
for autotext in autotexts:
    autotext.set_color('black')
    autotext.set_fontweight('bold')

# 4. Gráfico de barras horizontal ordenado
ax4 = axes[1, 1]
distribucion_sorted = distribucion.sort_values(ascending=True)
distribucion_sorted.plot(kind='barh', ax=ax4, color='mediumseagreen', edgecolor='black')
ax4.set_title('Distribución Ordenada (de Menor a Mayor)', fontweight='bold')
ax4.set_xlabel('Cantidad de Registros', fontweight='bold')
ax4.set_ylabel('Clase de Obesidad', fontweight='bold')
ax4.grid(axis='x', alpha=0.3)

# Agregar valores en las barras
for i, v in enumerate(distribucion_sorted.values):
    ax4.text(v + 10, i, str(v), ha='left', va='center', fontweight='bold')

plt.tight_layout()

# Guardar la figura
nombre_archivo = 'results/figuras/01_distribucion_clases_obesidad.png'
import os
os.makedirs('results/figuras', exist_ok=True)
plt.savefig(nombre_archivo, dpi=300, bbox_inches='tight')
print(f"✓ Visualizaciones guardadas en: {nombre_archivo}")

# Mostrar el gráfico
plt.show()


## 6. Generación de Reporte de Caracterización

### ¿Qué hace este paso?

Creamos un reporte en texto que documenta todos los hallazgos del análisis exploratorio. Este reporte es importante porque:

1. **Documentación**: Registra los hallazgos de forma permanente
2. **Comunicación**: Puede ser compartido con otros investigadores o stakeholders
3. **Referencia**: Útil para consultar más adelante sin ejecutar el código

### Contenido del Reporte

El reporte incluye:
- Información general del dataset
- Distribución detallada de cada clase
- Análisis de desbalance con métricas
- Problemas identificados
- Recomendaciones para el siguiente paso

### ¿Por qué guardar en archivo?

Aunque el notebook es interactivo, tener un archivo de texto permite:
- Consultar rápidamente sin abrir el notebook
- Incluir en documentación externa
- Compartir con personas que no tienen acceso al notebook


In [ ]:
# Crear el reporte
orden_ordinal = [
    'Insufficient_Weight',
    'Normal_Weight', 
    'Overweight_Level_I',
    'Overweight_Level_II',
    'Obesity_Type_I',
    'Obesity_Type_II',
    'Obesity_Type_III'
]

reporte = f"""
{'='*80}
REPORTE DE CARACTERIZACIÓN DEL DATASET
Problema: Regresión Ordinal - Clasificación de Niveles de Obesidad
{'='*80}

1. INFORMACIÓN GENERAL DEL DATASET
   - Total de registros: {len(df)}
   - Total de columnas: {len(df.columns)}
   - Variable objetivo: {target_column}
   - Número de clases: {len(distribucion)}

2. DISTRIBUCIÓN DE CLASES DE OBESIDAD
   (Ordenadas según su orden ordinal)

   Clase                    | Cantidad | Porcentaje | Estado
   {'-'*70}
"""

for clase in orden_ordinal:
    if clase in distribucion.index:
        cantidad = distribucion[clase]
        porcentaje = porcentajes[clase]
        
        # Clasificar el estado de la clase
        if porcentaje < 10:
            estado = "CRÍTICA (muy pocos datos)"
        elif porcentaje < 15:
            estado = "BAJA"
        elif porcentaje < 25:
            estado = "MODERADA"
        else:
            estado = "ALTA"
        
        reporte += f"   {clase:24} | {cantidad:8} | {porcentaje:9.2f}% | {estado}\n"

reporte += f"""
3. ANÁLISIS DE DESBALANCE

   Clase Mayoritaria: {distribucion.idxmax()}
   - Cantidad: {clase_mayoritaria} registros
   - Porcentaje: {porcentajes[distribucion.idxmax()]:.2f}%

   Clase Minoritaria: {distribucion.idxmin()}
   - Cantidad: {clase_minoritaria} registros
   - Porcentaje: {porcentajes[distribucion.idxmin()]:.2f}%

   Ratio de Desbalance (IR): {imbalance_ratio:.2f}
   Nivel de Desbalance: {nivel}

4. PROBLEMAS IDENTIFICADOS

   ✓ La caracterización de la base de datos ahora está completa
   ✓ Se ha identificado el problema de desbalance:
     - El ratio de desbalance es {imbalance_ratio:.2f}
     - Esto indica que hay una diferencia significativa entre las clases
     - Las clases con menos datos pueden tener peor rendimiento en el modelo

5. RECOMENDACIONES

   - Implementar técnicas de balanceo de datos (SMOTE, ADASYN, etc.)
   - Usar métricas de evaluación apropiadas para datos desbalanceados
   - Considerar pesos de clase en los modelos
   - Usar validación cruzada estratificada
   - Considerar modelos específicos para regresión ordinal

{'='*80}
Reporte generado automáticamente
{'='*80}
"""

# Guardar el reporte
os.makedirs('results/reportes', exist_ok=True)
nombre_reporte = 'results/reportes/01_reporte_caracterizacion.txt'
with open(nombre_reporte, 'w', encoding='utf-8') as f:
    f.write(reporte)

print(f"✓ Reporte guardado en: {nombre_reporte}")

# Mostrar el reporte
print(reporte)
